# MLOps with `vetiver`

## Build a model
Data scientists can still use the tools they are most comfortable with for the bulk of their workflow.

In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, pipeline
from sklearn.ensemble import RandomForestRegressor

from vetiver import VetiverModel, VetiverAPI, vetiver_endpoint

np.random.seed(500)

raw = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-03-02/youtube.csv')
df = pd.DataFrame(raw)

In [2]:
df = df[["like_count", "funny", "show_product_quickly", "patriotic", \
    "celebrity", "danger", "animals"]].dropna()
X, y = df.iloc[:,1:],df['like_count']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,test_size=0.2)

le = preprocessing.OrdinalEncoder().fit(X)
rf = RandomForestRegressor().fit(le.transform(X_train), y_train)

In [3]:
rf = pipeline.Pipeline([('label_encoder',le), ('random_forest', rf)])

## Deploy a model

Users first create a deployable model.


*In R, you saw the equivalent, `vetiver_model()`.*

In [5]:
v = VetiverModel(rf, save_ptype = True, ptype_data=X_train, model_name = "superbowl_rf")

Next, users initialize their API.

*In R, you saw the equivalents, `vetiver_api()` and `pr_run()`.*

In [6]:
app = VetiverAPI(v, check_ptype=True)

app.run()

/Users/isabelzimmerman/code/github/vetiver-python/vetiver/utils.py:14: UserWarning: WARNING: Jupyter Notebooks are not considered stable environments for production code
  warnings.warn(
INFO:     Started server process [34377]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [34377]


In [7]:
from rsconnect.api import RSConnectServer
from rsconnect.actions import deploy_python_fastapi
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

api_key = os.getenv("API_KEY")
rsc_url = os.getenv("RSC_URL")
connect_server = RSConnectServer(url = rsc_url, api_key = api_key)

In [ ]:
# deploy_python_fastapi(
#     connect_server = connect_server,
#     directory = ".",
#     extra_files = None,
#     entry_point = "superbowlads:api",
#     excludes = None,
# )

In [8]:
from vetiver.server import predict, vetiver_endpoint
data = {
  "funny": False,
  "show_product_quickly": True,
  "patriotic": True,
  "celebrity": False,
  "danger": True,
  "animals": True
}
endpoint = vetiver_endpoint("https://colorado.rstudio.com/rsc/content/e5a6dc06-4839-469f-b6dd-2fc03a8abf7f/predict/")

response = predict(data = data, endpoint = endpoint)

In [9]:
response

{'prediction': 9558.841252136752}